In [1]:
import os

import imageio
import torch

# 1
## (a)

In [13]:
batch_size = 4
batch = torch.zeros(batch_size, 3, 4032, 3024, dtype=torch.uint8)

In [15]:
data_dir = "data/my_picture/"
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == ".jpeg"]
for i, filename in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr.copy())
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3] # 余計な次元を削除する
    batch[i] = img_t

In [16]:
batch

tensor([[[[178, 181, 178,  ...,  53,  55,  53],
          [178, 181, 179,  ...,  53,  60,  62],
          [168, 171, 175,  ...,  50,  64,  70],
          ...,
          [ 19,  20,  19,  ..., 185, 186, 190],
          [ 18,  19,  19,  ..., 185, 188, 191],
          [ 14,  17,  18,  ..., 184, 187, 190]],

         [[123, 123, 120,  ...,  22,  24,  22],
          [123, 123, 121,  ...,  22,  29,  31],
          [113, 113, 117,  ...,  19,  33,  39],
          ...,
          [ 21,  22,  21,  ..., 125, 128, 132],
          [ 20,  21,  21,  ..., 127, 130, 133],
          [ 16,  19,  20,  ..., 128, 131, 134]],

         [[ 59,  60,  57,  ...,   0,   0,   0],
          [ 59,  60,  58,  ...,   0,   0,   3],
          [ 49,  50,  54,  ...,   0,   2,  10],
          ...,
          [ 20,  21,  20,  ...,  62,  64,  68],
          [ 19,  20,  20,  ...,  64,  67,  70],
          [ 15,  18,  19,  ...,  67,  70,  73]]],


        [[[165, 168, 173,  ..., 252, 253, 252],
          [166, 169, 173,  ..., 252

In [20]:
batch.shape

torch.Size([4, 3, 4032, 3024])

## (b)

In [24]:
torch.mean(batch.float(), dim=(1, 2, 3))

tensor([103.1541, 115.7505, 168.2598, 151.4030])

## (c)  
画像は一枚目から青緑青赤の順番

In [25]:
torch.mean(batch.float(), dim=(2, 3))

tensor([[ 88.4817, 102.8620, 118.1186],
        [121.0544, 116.5072, 109.6898],
        [166.3025, 171.7381, 166.7390],
        [166.8059, 148.9472, 138.4558]])

R G Bの順  
今回の例に関しては、平均では物体を識別できない  
画像に他の物体が混ざっていることもあるし、今回私が撮った物体が完全な青や緑というよりはくすんでいたりするので、その結果うまくいかなかったと思われる。

# 2  
ファイルはYOLOv5のtrain.pyを用いる  
## (a)

In [26]:
with open("data/yolo.py", encoding="utf8") as f:
    text = f.read()

In [37]:
import re

def clean_words(input_str):
    word_list = input_str.lower().replace("\n", " ").split()
    word_list = [re.sub(r"[^a-zA-Z0-9_]+", " ", word).split() for word in word_list]
    return sum(word_list, [])

In [38]:
clean_words(text)

['yolov5',
 'by',
 'ultralytics',
 'gpl',
 '3',
 '0',
 'license',
 'train',
 'a',
 'yolov5',
 'model',
 'on',
 'a',
 'custom',
 'dataset',
 'models',
 'and',
 'datasets',
 'download',
 'automatically',
 'from',
 'the',
 'latest',
 'yolov5',
 'release',
 'models',
 'https',
 'github',
 'com',
 'ultralytics',
 'yolov5',
 'tree',
 'master',
 'models',
 'datasets',
 'https',
 'github',
 'com',
 'ultralytics',
 'yolov5',
 'tree',
 'master',
 'data',
 'tutorial',
 'https',
 'github',
 'com',
 'ultralytics',
 'yolov5',
 'wiki',
 'train',
 'custom',
 'data',
 'usage',
 'python',
 'path',
 'to',
 'train',
 'py',
 'data',
 'coco128',
 'yaml',
 'weights',
 'yolov5s',
 'pt',
 'img',
 '640',
 'from',
 'pretrained',
 'recommended',
 'python',
 'path',
 'to',
 'train',
 'py',
 'data',
 'coco128',
 'yaml',
 'weights',
 'cfg',
 'yolov5s',
 'yaml',
 'img',
 '640',
 'from',
 'scratch',
 'import',
 'argparse',
 'import',
 'math',
 'import',
 'os',
 'import',
 'random',
 'import',
 'sys',
 'import',
 'time

In [41]:
word_list = sorted(set(clean_words(text)))
word2index_dict = {word: i for (i, word) in enumerate(word_list)}

In [42]:
word2index_dict

{'0': 0,
 '001': 1,
 '01': 2,
 '02': 3,
 '1': 4,
 '10': 5,
 '100': 6,
 '1000': 7,
 '10b': 8,
 '10s': 9,
 '16': 10,
 '1e': 11,
 '1e9': 12,
 '1k': 13,
 '2': 14,
 '25': 15,
 '255': 16,
 '3': 17,
 '300': 18,
 '32': 19,
 '320': 20,
 '3600': 21,
 '3f': 22,
 '3g': 23,
 '4': 24,
 '45': 25,
 '4576': 26,
 '475': 27,
 '4g': 28,
 '5': 29,
 '50': 30,
 '6': 31,
 '60': 32,
 '64': 33,
 '640': 34,
 '65': 35,
 '7': 36,
 '75': 37,
 '8': 38,
 '80': 39,
 '9': 40,
 '95': 41,
 '98': 42,
 '999': 43,
 '_': 44,
 '__file__': 45,
 '__main__': 46,
 '__name__': 47,
 '_initialize_biases': 48,
 'a': 49,
 'accumulate': 50,
 'action': 51,
 'actions': 52,
 'adam': 53,
 'adamw': 54,
 'add': 55,
 'add_argument': 56,
 'add_param_group': 57,
 'additional': 58,
 'adjust': 59,
 'align_corners': 60,
 'all': 61,
 'amp': 62,
 'an': 63,
 'anchor': 64,
 'anchor_t': 65,
 'anchors': 66,
 'and': 67,
 'any': 68,
 'append': 69,
 'are': 70,
 'argparse': 71,
 'argsort': 72,
 'argument': 73,
 'argumentparser': 74,
 'arguments': 75,
 'arra

## (b)

In [43]:
len(word2index_dict)

818

「高慢と偏見」の方がサイズが大きい

## (c)

In [45]:
lines = text.split("\n")
line = lines[50]
words_in_line = clean_words(line)

In [46]:
word_t = torch.zeros(len(words_in_line), len(word2index_dict))
for i, word in enumerate(words_in_line):
    word_index = word2index_dict[word]
    word_t[i][word_index] = 1
    print("{:2} {:4} {}".format(i, word_index, word))
    
print(word_t.shape)

 0  386 intersect_dicts
 1  413 labels_to_class_weights
 2  414 labels_to_image_weights
 3  468 methods
 4  545 one_cycle
torch.Size([5, 818])


## (d)
高慢と偏見では、句読点(カンマコンマ)の情報が失われ、文の区切りがわかりにくくなっていたが、文末に同じ単語がある可能性が高いことを考慮すれば、それほど大きな影響とは言えなかった。  
一方、pythonファイルでは、:や()\[\]などのpythonの文法に大きな影響を与える文字が削除されてしまうため、ファイルの構造について区切りなどの情報がかなり大きく失われてしまう。